In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
from keras.datasets import mnist
from sklearn.metrics import accuracy_score


In [ ]:
# Налаштування параметрів нейромережі
num_classes = 10  # Кількість класів (цифри від 0 до 9)
num_features = 784  # Кількість ознак (28x28 пікселів = 784)

learning_rate = 0.001  # Швидкість навчання
training_steps = 3000  # Кількість ітерацій навчання
batch_size = 256  # Розмір пакету
display_step = 100  # Як часто виводити інформацію про прогрес

# Розмірність шарів мережі
n_hidden_1 = 128  # Кількість нейронів у першому прихованому шарі
n_hidden_2 = 256  # Кількість нейронів у другому прихованому шарі

# Завантажуємо набір даних MNIST (рукописні цифри)
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Перетворюємо дані в формат float32 та нормалізуємо їх у діапазон [0,1]
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
x_train, x_test = x_train / 255., x_test / 255.  # Нормалізація

# Створюємо об'єкт TensorFlow Dataset для ефективної обробки даних
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [ ]:
# Оголошуємо клас для повнозв'язного шару
class DenseLayer(tf.Module):
    def __init__(self, in_features, out_features, activation=None, name=None):
        super().__init__(name=name)
        self.w = tf.Variable(tf.random.normal([in_features, out_features]), name="w")  # Випадкові ваги
        self.b = tf.Variable(tf.zeros([out_features]), name="b")  # Початкові нулі для зміщення (bias)
        self.activation = activation  # Функція активації

    def __call__(self, x):
        y = tf.matmul(x, self.w) + self.b  # Лінійне перетворення
        return self.activation(y) if self.activation else y  # Застосовуємо активацію, якщо є

# Оголошуємо клас нейромережі
class NN(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.layer1 = DenseLayer(num_features, n_hidden_1, activation=tf.nn.sigmoid)
        self.layer2 = DenseLayer(n_hidden_1, n_hidden_2, activation=tf.nn.sigmoid)
        self.out_layer = DenseLayer(n_hidden_2, num_classes, activation=tf.nn.softmax)  # Softmax для багатокласової класифікації

    def __call__(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return self.out_layer(x)

# Функція крос-ентропії для обчислення втрат
def cross_entropy(y_pred, y_true):
    y_true = tf.one_hot(y_true, depth=num_classes)  # Перетворюємо y_true у one-hot представлення
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)  # Уникаємо логарифма від 0
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred), axis=1))  # Обчислення втрат

# Функція для обчислення точності моделі
def accuracy(y_pred, y_true):
    return accuracy_score(y_true, np.argmax(y_pred, axis=1))  # Порівнюємо прогноз із реальними значеннями

# Створюємо екземпляр нейромережі
neural_net = NN(name="mnist")

# Функція навчання моделі
def train(neural_net, input_x, output_y):
    optimizer = tf.optimizers.SGD(learning_rate)  # Стохастичний градієнтний спуск (SGD)
    with tf.GradientTape() as g:
        pred = neural_net(input_x)  # Прогноз мережі
        loss = cross_entropy(pred, output_y)  # Обчислення помилки
    # Обчислення градієнтів для кожного шару
    gradients = g.gradient(loss, [neural_net.layer1.w, neural_net.layer1.b,
                                  neural_net.layer2.w, neural_net.layer2.b,
                                  neural_net.out_layer.w, neural_net.out_layer.b])
    # Оновлення ваг
    optimizer.apply_gradients(zip(gradients, [neural_net.layer1.w, neural_net.layer1.b,
                                              neural_net.layer2.w, neural_net.layer2.b,
                                              neural_net.out_layer.w, neural_net.out_layer.b]))
    return loss

In [ ]:

# Історія втрат та точності
loss_history = []
accuracy_history = []

# Процес навчання
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    loss = train(neural_net, batch_x, batch_y)  # Навчаємо модель на пакеті даних
    if step % display_step == 0:
        pred = neural_net(batch_x)  # Робимо прогноз
        acc = accuracy(pred.numpy(), batch_y.numpy())  # Обчислюємо точність
        loss_history.append(loss.numpy())
        accuracy_history.append(acc)
        print(f"Крок {step}, Втрата: {loss.numpy():.4f}, Точність: {acc:.4f}")


In [ ]:
# Візуалізація процесу навчання
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(loss_history, label='Втрати')
plt.xlabel("Кроки (x100)")
plt.ylabel("Втрати")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(accuracy_history, label='Точність', color='red')
plt.xlabel("Кроки (x100)")
plt.ylabel("Точність")
plt.legend()
